<a href="https://colab.research.google.com/github/muirjc/PAExample/blob/main/MilvusLiteBookExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install milvus with [client] extras by pip
!python3 -m pip install milvus
!python3 -m pip install milvus[client]

In [2]:
from milvus import default_server
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

default_server.cleanup
default_server.start()
print("port: ", default_server.listen_port)

connections.connect(host='localhost', port=default_server.listen_port)
print(utility.get_server_version())



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.11-lite
 Process:   35300
 Started:   2023-07-11 21:31:45
 Config:    /root/.milvus.io/milvus-server/2.2.11/configs/milvus.yaml
 Logs:      /root/.milvus.io/milvus-server/2.2.11/logs

 Ctrl+C to exit ...
port:  19530


v2.2.11-lite


In [3]:
book_id = FieldSchema(name="book_id", dtype=DataType.INT64, is_primary=True,)
book_name = FieldSchema(name="book_name", dtype=DataType.VARCHAR, max_length=200,)
word_count = FieldSchema(name="word_count", dtype=DataType.INT64,)
book_intro = FieldSchema(name="book_intro", dtype=DataType.FLOAT_VECTOR, dim=2)

schema = CollectionSchema(
  fields=[book_id, book_name, word_count, book_intro],
  description="Test book search",
  enable_dynamic_field=True
)

collection_name = "book"

collection = Collection(name=collection_name, schema=schema)

In [4]:
import random
data = [
  [i for i in range(2000)],
  [str(i) for i in range(2000)],
  [i for i in range(10000, 12000)],
  [[random.random() for _ in range(2)] for _ in range(2000)]
]

## Once your collection is enabled with dynamic schema,
## you can add non-existing field values.
data.append([str("dy"*i) for i in range(2000)])

book_id = data[0]
book_name = data[1]
word_count = data[2]
book_intro = data[3]

data = []
data.append(book_id)
data.append(book_name)
data.append(word_count)
data.append(book_intro)

In [5]:
mr = collection.insert(data)

In [6]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}

collection.create_index(
  field_name="book_intro",
  index_params=index_params
)

Status(code=0, message=)

In [7]:
utility.wait_for_index_building_complete(collection_name)

True

In [8]:
collection.create_index(
  field_name="book_name",
  index_name="scalar_index",
)

Status(code=0, message=)

In [9]:
collection.load()

In [10]:
utility.load_state("book")

<LoadState: Loaded>

In [11]:
utility.loading_progress("book")

{'loading_progress': '100%'}

In [12]:
res = collection.query(
  expr = "book_id in [2,4,6,8]",
  offset = 0,
  limit = 10,
  output_fields = ["book_id", "book_intro"],
  consistency_level="Strong"
)

print(res)

[{'book_id': 2, 'book_intro': [0.045682855, 0.41438448]}, {'book_id': 4, 'book_intro': [0.24060236, 0.9916207]}, {'book_id': 6, 'book_intro': [0.9197221, 0.3926468]}, {'book_id': 8, 'book_intro': [0.30012918, 0.7561632]}]
